In [ ]:
# default_exp pipeline

# Pipeline

> Custom data pipeline, including transformers, dataloaders and data augmentations.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
import torch
from scipy.ndimage import zoom
from fastai2.torch_core import TensorImage, Tuple
from fastcore.transform import ItemTransform, compose_tfms
from fastai2.data.block import TransformBlock, DataBlock, Transform, ColSplitter, ColReader

In [ ]:
#exports
class SequenceTfms(ItemTransform):
    """ Custom Transformer for 2D or 3D data that applies same augmentations to X and y """

    def __init__(self, tfms):
        self.tfms = tfms
        self.split_idx = 0

    def encodes(self, pair):
        # 3D
        if pair[0].shape[0] > 3:
            pair[0] = pair[0].squeeze(1)
            pair[1] = pair[1].squeeze(1)
            seq_len, rs, cs = pair[0].shape
            x = pair[0].view(1, seq_len, rs, cs)
            y = pair[1].view(1, seq_len, rs, cs)
            z = torch.cat([x, y], dim=1)
            z = TensorImage(z)
            z = compose_tfms(z, self.tfms)
            x = z[:1, :seq_len]
            y = z[:1, seq_len:]
            x = x.view(seq_len, 1, rs, cs)
            y = y.view(seq_len, 1, rs, cs)
        # 2D
        else:
            seq_len, rs, cs = pair[0].shape
            x = pair[0].unsqueeze(0)
            y = pair[1].unsqueeze(0)
            z = torch.cat([x, y], dim=1)
            z = TensorImage(z)
            z = compose_tfms(z, self.tfms)
            x = z[:1, :seq_len]
            y = z[:1, seq_len:]
            x = x.squeeze(0)
            y = y.squeeze(0)

        x = TensorImage(x)
        y = TensorImage(y)

        return x, y

In [ ]:
#exports
class ImageSequence(Tuple):
    """ Custom data loader for 2d or 3d data from numpy arrays """

    def __init__(self, scaling=(1, 1, 1)):
        self.scaling = scaling

    def create(self, file):

        img = np.load(file, allow_pickle=True)

        # resizing
        if len(img.shape) > 2 and img.shape[-1] > 3:  # 3D
            img = zoom(img, self.scaling, order=2)
        elif len(img.shape) > 2 and img.shape[-1] <= 3:  # 2.5D
            img = zoom(img, self.scaling[:2] + (1,), order=2)
        else:
            img = zoom(img, self.scaling[:2], order=2)  # 2D

        if len(img.shape) > 2:
            if img.shape[-1] > 3:
                return TensorImage(img.swapaxes(2, 1).swapaxes(1, 0)).unsqueeze(1)  # 3D
            else:
                return TensorImage(img.swapaxes(2, 1).swapaxes(1, 0))  # 2.5D
        return TensorImage(img[None,])  # 2D

In [ ]:
#exports
def ImageSequenceBlockWrapper(scaling=(1, 1, 1), tfms=[]):
    """ Custom transform block (wrapped)"""
    def ImageSequenceBlock():
        """ Custom transform block """
        return TransformBlock(type_tfms=[ImageSequence(scaling).create, ],
                              item_tfms=[SequenceTfms([*tfms])],
                              batch_tfms=[])
    return ImageSequenceBlock

In [ ]:
#exports
class SwapTfms(Transform):
    """ Just a transform for swapping the channel dimension to the front """
    def __init__(self):
        pass

    def encodes(self, x: TensorImage):
        if len(x.size()) < 5:
            return x
        return x.permute(0, 2, 1, 3, 4)

In [ ]:
#exports
def get_dls(df, scaling=(1,1,1), bs=8, tfms=[], out_ch=1):
    """ Data-loader pipeline and transforms for either 2D, 2.5D or 3D models """

    # data-block API using the custom pipelines and reading the location of the training files from a dataframe
    dblock = DataBlock(blocks=(ImageSequenceBlockWrapper(scaling, tfms), ImageSequenceBlockWrapper(scaling, tfms)),
                       splitter=ColSplitter(),
                       get_x=ColReader('X'),
                       get_y=ColReader('y'),
                       item_tfms=[],
                       batch_tfms=[SwapTfms()])

    dls = dblock.dataloaders(df, bs=bs)
    dls.c = out_ch  # number of channels of the target

    print(f"{dls.one_batch()[0].shape} - Scaling: {scaling}")

    return dls